In [1]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from scipy import constants as sc
import scipy as sp
import numpy as np

%matplotlib inline

from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from IPython.display import display
from ipywidgets import *
import matplotlib as mpl

mpl.rcParams.update({'font.size': 14})
mpl.rc('font', family='Arial',weight='bold')
mpl.rc('savefig',dpi=600)

from mayavi import mlab
#mlab.init_notebook()
%gui qt

In [2]:
def full(w,hideDes=False):
    for i in w.children:
        i.layout.width='100%'
        if hideDes:
            i.description = ''
    return w

def doIT(f,*args,**kwargs):
    display(full(interactive(f,*args,**kwargs)))

In [3]:
class Crystal():
    __fname = ''
    __cp  = []   #cell positions
    __ap  = []   #atom positions
    __Ns  = 0    #Atomic Species Count
    __SP  = []   #Atomic Species Names
    __Runs= 0    #Number of Runs

    
    def __init__(self,fname):
        self.loadFile(fname)
        
    def getFname(self):
        return self.__fname
    
    def loadFile(self,fname):
        self.__fname=fname
        with open(fname,'r') as f:
            s = f.readlines()
        sp = [i.split() for i in s]
        cind = [i for i,l in enumerate(s) if 'CELL_PARAMETERS' in l]
        aind = [i for i,l in enumerate(s) if 'ATOMIC_POSITIONS' in l]

        def getNS(ai):
            count = 0
            while True:
                if len(s[ai+count+1].split())==0:
                    break
                count += 1
                return count
        N = getNS(aind[0])
        self.__Ns = N
        self.__SP = np.array(sp[aind[0]+1:aind[0]+1+N])[:,0]
        self.__cp = np.array([np.array(sp[i+1:j-1]) for i,j in zip(cind,aind)]).astype(np.float)
        self.__ap = np.array([np.array(sp[i+1:i+2+N]) for i in aind])[:,:,1:].astype(np.float)
        self.__Runs=len(cind)
    
    def mkCrystalPoints(self,Run,N):
        mult = lambda A,r: (r*A.transpose()).transpose()
        R = lambda h,k,l,A: h*A[0]+k*A[1]+l*A[2]
        Rs = lambda h,k,l,A,r: np.sum(mult(np.array(A),np.array([h,k,l]))+mult(np.array(A),np.array(r)),axis=0)

        Points = lambda CP,SP,N: np.array([Rs(i,j,k,CP,SP) 
                                         for i in range(N) 
                                         for j in range(N) 
                                         for k in range(N)]).transpose()
        count = 1
        
        x,y,z,c=[],[],[],[]
        #x,y,z = Points(self.__cp[Run],np.zeros(3),N)
        #c = np.ones(len(x))*0
        for i in self.__ap[Run]:
            Ix,Iy,Iz = Points(self.__cp[Run],i,N)
            Ic = np.ones(len(Ix))*count
            x = np.concatenate((x,Ix),axis=0)
            y = np.concatenate((y,Iy),axis=0)
            z = np.concatenate((z,Iz),axis=0)
            c = np.concatenate((c,Ic),axis=0)
            count +=1
        #pts = mlab.points3d(x,y,z,c,colormap='jet',scale_factor=1)
        sV = np.ones(len(x))
        return x,y,z,c,sV
    
    def plotCrystal(self,Run,N,scale=.5):
        mlab.clf()
        x,y,z,c,sV = self.mkCrystalPoints(Run,N)
        pts = mlab.quiver3d(x, y, z, sV, sV, sV, scalars=c, mode="sphere"
                            , scale_factor=scale,vmin=0,colormap='jet')
        pts.glyph.color_mode = "color_by_scalar"
        pts.glyph.glyph_source.glyph_source.center = [0,0,0]
        print type(pts)
        return pts
        #mlab.show()
    
    def getNumberRuns(self):
        return self.__Runs
    
    def getPositions(self):
        return self.__cp,self.__ap

In [4]:
fname = 'InAs_Bulk_PAW.in_ecut_50.00Ry.out'
c = Crystal(fname)

In [5]:
cp,ap = c.getPositions()

In [29]:
print cp
#print np.sum(cp[0],axis=0)

[[[  3.08516379e+00   3.08516379e+00  -3.84750000e-05]
  [  3.08451103e+00  -8.20930000e-05   3.08452497e+00]
  [ -8.20930000e-05   3.08451103e+00   3.08452497e+00]]

 [[  3.08848384e+00   3.08848384e+00  -5.32880000e-05]
  [  3.08788132e+00  -1.47892000e-04   3.08791433e+00]
  [ -1.47892000e-04   3.08788132e+00   3.08791433e+00]]

 [[  3.09345583e+00   3.09345583e+00  -4.27390000e-05]
  [  3.09294312e+00  -2.28275000e-04   3.09301476e+00]
  [ -2.28275000e-04   3.09294312e+00   3.09301476e+00]]

 [[  3.09457238e+00   3.09457238e+00  -8.49200000e-06]
  [  3.09409673e+00  -2.31359000e-04   3.09418841e+00]
  [ -2.31359000e-04   3.09409673e+00   3.09418841e+00]]

 [[  3.09457238e+00   3.09457238e+00  -8.49200000e-06]
  [  3.09409673e+00  -2.31359000e-04   3.09418841e+00]
  [ -2.31359000e-04   3.09409673e+00   3.09418841e+00]]]


In [18]:
doIT(c.plotCrystal,Run=(0,c.getNumberRuns()-1),N=(1,14),scale=(.1,1.))

<class 'mayavi.modules.vectors.Vectors'>


In [19]:
print np.std(ap,axis=0)#/np.mean(ap,axis=0)*100

[[  1.00098122e-05   3.50402571e-06   3.50402571e-06]
 [  1.00098122e-05   3.50402571e-06   3.50402571e-06]]


In [14]:
print np.std(cp,axis=0)/np.mean(cp,axis=0)*100

[[  0.12244629   0.12244629 -60.88191427]
 [  0.12477086 -32.69568464   0.12577943]
 [-32.69568464   0.12477086   0.12577943]]
